# PAIRS query to download data required for tutorial 3

# Preparatory steps

### Toggle here to run on Watson Studio or locally

In [1]:
running_watson_studio=False

### Set up Watson studio project token - replace project ids and tokens for your Watson Studio project as described in workshop setup instructions [here](https://github.com/C2MA-workshop/c2ma-docs)

In [2]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
if running_watson_studio:
    from project_lib import Project
    project = Project(project_id='XXXX', project_access_token='XXXX')
    pc = project.project_context

### Install the PAIRS api library  

In [3]:
!pip install ibmpairs

### Load the required libraries  

In [4]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Retrieve time series data from PAIRS 

### Connect to PAIRS - Watson Studio version
You should already have copied your api key to your Watson Studio project following the setup instrictions [here](https://github.com/C2MA-workshop/c2ma-docs).

If not please do so now, then return to this tutorial.

### PAIRS authentication in Watson Studio

In [5]:
if running_watson_studio:
    from ibmpairs import paw, authentication
    my_file = project.get_file("pairspass.txt") 
    PAIRS_API_KEY=my_file.readline().decode('utf-8') 
    PAIRS_SERVER = "https://pairs.res.ibm.com"
    OAUTH = authentication.OAuth2(api_key = PAIRS_API_KEY )

### PAIRS authentication - local version

In [6]:
# Local version
if not running_watson_studio:
    from ibmpairs import paw, authentication 
    with open("/Users/annejones/pairspass.txt") as my_file: #change to location of your pairspass.txt file
        PAIRS_API_KEY=my_file.readline()
        PAIRS_SERVER = "https://pairs.res.ibm.com"
        OAUTH = authentication.OAuth2(api_key = PAIRS_API_KEY )

### Create dictionary to store the layer ids we may want to use

In [7]:
pairs_dict = {
    'ERA5 temperature' : 49423,
    'ERA5 rainfall' : 49459,
    'SEDAC population' : 48774
}

### Select a location for which we want to run the model

In [9]:
lat1 = -24.99
lon1 = 31.59

lat2 = -22.97
lon2 = 30.50

lat3 = -22.27
lon3 = 29.90

locations = [str(lat1), str(lon1), str(lat2), str(lon2), str(lat3), str(lon3)]

### Create point query json for two layers simultaneously
Query all the locations, for 20 years

In [10]:
query_json = {
      "layers" : [
          {"type" : "raster", "id" : pairs_dict["ERA5 temperature"]},
          {"type" : "raster", "id" : pairs_dict["ERA5 rainfall"]}
      ],
      "spatial" : {"type" : "point",  "coordinates" : locations}, 
      "temporal" : {"intervals" : [
          {"start" : "2001-01-01T00:00:00Z", "end" : "2021-01-01T00:00:00Z"}
      ]}
  }

In [11]:
# create query object
query = paw.PAIRSQuery(query_json, PAIRS_SERVER,  auth=OAUTH, authType='api-key') 
# submit the query
query.submit()

In [12]:
# check the data returned
query.vdf.head()

,layerId,timestamp,longitude,latitude,value,region,property,geometry
0,49423,2001-01-01 01:00:00+00:00,31.59,-24.99,294.261505,NaN,NaN,POINT (31.59000 -24.99000)
1,49423,2001-01-01 01:00:00+00:00,30.50,-22.97,293.807953,NaN,NaN,POINT (30.50000 -22.97000)
2,49423,2001-01-01 01:00:00+00:00,29.90,-22.27,297.289764,NaN,NaN,POINT (29.90000 -22.27000)
3,49423,2001-01-01 02:00:00+00:00,31.59,-24.99,294.341980,NaN,NaN,POINT (31.59000 -24.99000)
4,49423,2001-01-01 02:00:00+00:00,30.50,-22.97,293.883362,NaN,NaN,POINT (30.50000 -22.97000)


# Post-processing of the queried data

### Add variable name to the dataframe for ease of use

In [13]:
query.vdf['var'] = None
query.vdf.loc[query.vdf['layerId']==pairs_dict["ERA5 temperature"], 'var'] = 'temperature'
query.vdf.loc[query.vdf['layerId']==pairs_dict["ERA5 rainfall"], 'var'] = 'rainfall'

### Add consituents of date - day, month, year 

In [14]:
query.vdf['day'] = query.vdf['timestamp'].dt.day
query.vdf['month'] = query.vdf['timestamp'].dt.month
query.vdf['year'] = query.vdf['timestamp'].dt.year
query.vdf.head()

,layerId,timestamp,longitude,latitude,value,region,property,geometry,var,day,month,year
0,49423,2001-01-01 01:00:00+00:00,31.59,-24.99,294.261505,NaN,NaN,POINT (31.59000 -24.99000),temperature,1,1,2001
1,49423,2001-01-01 01:00:00+00:00,30.50,-22.97,293.807953,NaN,NaN,POINT (30.50000 -22.97000),temperature,1,1,2001
2,49423,2001-01-01 01:00:00+00:00,29.90,-22.27,297.289764,NaN,NaN,POINT (29.90000 -22.27000),temperature,1,1,2001
3,49423,2001-01-01 02:00:00+00:00,31.59,-24.99,294.341980,NaN,NaN,POINT (31.59000 -24.99000),temperature,1,1,2001
4,49423,2001-01-01 02:00:00+00:00,30.50,-22.97,293.883362,NaN,NaN,POINT (30.50000 -22.97000),temperature,1,1,2001


### Transform units (temperature in degrees Celcius and rainfall in mm)

In [15]:
query.vdf.loc[query.vdf['var']=='temperature', 'value'] = query.vdf.loc[query.vdf['var']=='temperature', 'value'] - 273.15
query.vdf.loc[query.vdf['var']=='rainfall', 'value'] = query.vdf.loc[query.vdf['var']=='rainfall', 'value']*1000.0 # native units are m per hour

### Average from native hourly (ERA5) to daily data by aggregating the data frame

In [16]:
vars_agg = query.vdf.groupby(['layerId', 'day', 'month', 
                              'year', 'longitude', 'latitude', 'var'], 
                             as_index=False).aggregate('mean')
vars_agg

,layerId,day,month,year,longitude,latitude,var,value,region,property
0,49423,1,1,2001,29.90,-22.27,temperature,29.974467,NaN,NaN
1,49423,1,1,2001,30.50,-22.97,temperature,24.973081,NaN,NaN
2,49423,1,1,2001,31.59,-24.99,temperature,24.669413,NaN,NaN
3,49423,1,1,2002,29.90,-22.27,temperature,27.763040,NaN,NaN
4,49423,1,1,2002,30.50,-22.97,temperature,23.332628,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
43831,49459,31,12,2019,30.50,-22.97,rainfall,0.028727,NaN,NaN
43832,49459,31,12,2019,31.59,-24.99,rainfall,0.017131,NaN,NaN
43833,49459,31,12,2020,29.90,-22.27,rainfall,0.033201,NaN,NaN
43834,49459,31,12,2020,30.50,-22.97,rainfall,0.067392,NaN,NaN


### Convert rainfall to mm per day by multiplying by 24

In [17]:
vars_agg.loc[vars_agg['var']=='rainfall', 'value'] = vars_agg.loc[vars_agg['var']=='rainfall', 'value']*24.0

### Convert the dataframe to "wide" format i.e. multiple variables for the same date

In [18]:
cols_to_keep = ['day', 'month', 'year', 'longitude', 'latitude']
vars_wide = vars_agg[vars_agg['var']=='rainfall'].copy().reset_index(drop=True)
vars_wide.rename(columns = {'value': 'rainfall'}, inplace=True)
vars_wide.drop(columns=['var', 'region', 'property', 'layerId'], inplace=True)

vars_wide2 = vars_agg[vars_agg['var']=='temperature'].copy().reset_index(drop=True)
vars_wide2.rename(columns = {'value': 'temperature'}, inplace=True)
vars_wide2.drop(columns=['var', 'region', 'property','layerId'], inplace=True)

vars_wide = vars_wide.merge(vars_wide2, how = 'inner', on = ['day', 'month', 'year', 'longitude', 'latitude'])
df = vars_wide

In [19]:
df.head()

,day,month,year,longitude,latitude,rainfall,temperature
0,1,1,2001,29.90,-22.27,0.417360,29.974467
1,1,1,2001,30.50,-22.97,1.939459,24.973081
2,1,1,2001,31.59,-24.99,38.258316,24.669413
3,1,1,2002,29.90,-22.27,0.117372,27.763040
4,1,1,2002,30.50,-22.97,0.091737,23.332628


### Add a datetime corresponding to each day

In [20]:
def datetime_from_components(year, month, day):
    dt = np.datetime64(str(year) + '-' + str(month).zfill(2) + '-' + str(day).zfill(2))
    return dt

In [21]:
df['datetime'] = pd.to_datetime(df[['year', 'month', 'day']])

In [22]:
df.head()

,day,month,year,longitude,latitude,rainfall,temperature,datetime
0,1,1,2001,29.90,-22.27,0.417360,29.974467,2001-01-01
1,1,1,2001,30.50,-22.97,1.939459,24.973081,2001-01-01
2,1,1,2001,31.59,-24.99,38.258316,24.669413,2001-01-01
3,1,1,2002,29.90,-22.27,0.117372,27.763040,2002-01-01
4,1,1,2002,30.50,-22.97,0.091737,23.332628,2002-01-01


## Save data to file

### Watson Studio version (file appears in COS storage for your project)

In [27]:
if running_watson_studio:
    project.save_data(file_name = "sample_climate_data.csv",data = df.to_csv(index=False))

### Local version

In [26]:
if not running_watson_studio:
    df.to_csv("./sample-data/sample_climate_data.csv", index=False) # csv file (dates become strings)

### Author and license

Anne Jones is a Research Staff Member at IBM Research, specialising in AI for Climate Risk and Impacts. 

Copyright © 2021 IBM. This notebook and its source code are released under the terms of the MIT License.